In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [4]:
labelencoder = LabelEncoder()
encoded_y = labelencoder.fit_transform(Y)# you can use fit and transform functions sepratly also

In [5]:
print(encoded_y,X)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [[0.02   0.0371 0.0428 ... 0.0084 0.009  0.0032]
 [0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 ...
 [0.0522 0.0437 0.018  ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049  ... 0.0079 0.0036 0.0048]
 [0.026  0.0363 0.0136 ... 0.0036 0.0061 0.0115]]


In [6]:
from keras import models
from keras import layers
def create_baseline():
    model=models.Sequential()
    model.add(layers.Dense(8,activation='relu',input_shape=(60,)))
    model.add(layers.Dense(8,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

In [7]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

W0817 13:59:33.937127 140428981307200 deprecation_wrapper.py:119] From /home/mubashir/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 13:59:34.234521 140428981307200 deprecation_wrapper.py:119] From /home/mubashir/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 13:59:34.274602 140428981307200 deprecation_wrapper.py:119] From /home/mubashir/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 13:59:34.323146 140428981307200 deprecation_wrapper.py:119] From /home/mubashir/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Results: 83.14% (10.12%)


Re-Run The Baseline Model With Data Preparation

In [8]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 83.19% (6.43%)


In [9]:
#Tuning Layers and Number of Neurons in The Model

In [10]:
# smaller model
def create_smaller():
    model=models.Sequential()
    model.add(layers.Dense(60,activation='relu',input_shape=(60,)))
    model.add(layers.Dense(30,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 84.07% (5.85%)


In [11]:
# larger model
def create_larger():
    model=models.Sequential()
    model.add(layers.Dense(60,activation='relu',input_shape=(60,)))
    model.add(layers.Dense(60,activation='relu'))
    model.add(layers.Dense(40,activation='relu'))
    model.add(layers.Dense(20,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))

    model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 84.09% (3.84%)


step#5 Really Scaling up: developing a model that overfits

In [12]:
# overfitting my model
def create_larger():
    model=models.Sequential()
    model.add(layers.Dense(100,activation='relu',input_shape=(60,)))
    model.add(layers.Dense(100,activation='relu'))
    model.add(layers.Dense(60,activation='relu'))
    model.add(layers.Dense(50,activation='relu'))
    model.add(layers.Dense(35,activation='relu'))
    model.add(layers.Dense(20,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    # looks like i overfitted tooooo much
    model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=150, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/150
186/186 [==============================] - 4s 23ms/step - loss: 0.6287 - acc: 0.6344
Epoch 2/150
186/186 [==============================] - 0s 2ms/step - loss: 0.3371 - acc: 0.8710
Epoch 3/150
186/186 [==============================] - 0s 2ms/step - loss: 0.2246 - acc: 0.9140
Epoch 4/150
186/186 [==============================] - 0s 2ms/step - loss: 0.0650 - acc: 0.9892
Epoch 5/150
186/186 [==============================] - 0s 1ms/step - loss: 0.0269 - acc: 0.9946
Epoch 6/150
186/186 [==============================] - 0s 2ms/step - loss: 0.0082 - acc: 1.0000
Epoch 7/150
186/186 [==============================] - ETA: 0s - loss: 0.0026 - acc: 1.000 - 0s 2ms/step - loss: 0.0025 - acc: 1.0000
Epoch 8/150
186/186 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000
Epoch 9/150
186/186 [==============================] - 0s 2ms/step - loss: 9.4751e-04 - acc: 1.0000
Epoch 10/150
186/186 [==============================] - 0s 2ms/step - loss: 6.7405e-04 - acc:

186/186 [==============================] - 0s 2ms/step - loss: 4.1761e-06 - acc: 1.0000
Epoch 83/150
186/186 [==============================] - 0s 1ms/step - loss: 4.0470e-06 - acc: 1.0000
Epoch 84/150
186/186 [==============================] - 0s 2ms/step - loss: 3.9178e-06 - acc: 1.0000
Epoch 85/150
186/186 [==============================] - 0s 2ms/step - loss: 3.7952e-06 - acc: 1.0000
Epoch 86/150
186/186 [==============================] - 0s 2ms/step - loss: 3.6705e-06 - acc: 1.0000
Epoch 87/150
186/186 [==============================] - 0s 2ms/step - loss: 3.5641e-06 - acc: 1.0000
Epoch 88/150
186/186 [==============================] - 0s 2ms/step - loss: 3.4559e-06 - acc: 1.0000
Epoch 89/150
186/186 [==============================] - 0s 2ms/step - loss: 3.3490e-06 - acc: 1.0000
Epoch 90/150
186/186 [==============================] - 0s 2ms/step - loss: 3.2437e-06 - acc: 1.0000
Epoch 91/150
186/186 [==============================] - 0s 2ms/step - loss: 3.1405e-06 - acc: 1.0000
Epo

187/187 [==============================] - 0s 2ms/step - loss: 4.5925e-04 - acc: 1.0000
Epoch 13/150
187/187 [==============================] - 0s 1ms/step - loss: 3.6039e-04 - acc: 1.0000
Epoch 14/150
187/187 [==============================] - 0s 2ms/step - loss: 2.9331e-04 - acc: 1.0000
Epoch 15/150
187/187 [==============================] - 0s 2ms/step - loss: 2.4355e-04 - acc: 1.0000
Epoch 16/150
187/187 [==============================] - 0s 2ms/step - loss: 2.0443e-04 - acc: 1.0000
Epoch 17/150
187/187 [==============================] - 0s 2ms/step - loss: 1.7397e-04 - acc: 1.0000
Epoch 18/150
187/187 [==============================] - 0s 2ms/step - loss: 1.5215e-04 - acc: 1.0000
Epoch 19/150
187/187 [==============================] - 0s 2ms/step - loss: 1.2966e-04 - acc: 1.0000
Epoch 20/150
187/187 [==============================] - 0s 2ms/step - loss: 1.1421e-04 - acc: 1.0000
Epoch 21/150
187/187 [==============================] - 0s 2ms/step - loss: 1.0116e-04 - acc: 1.0000
Epo

187/187 [==============================] - 0s 2ms/step - loss: 2.1980e-06 - acc: 1.0000
Epoch 94/150
187/187 [==============================] - 0s 2ms/step - loss: 2.1337e-06 - acc: 1.0000
Epoch 95/150
187/187 [==============================] - 0s 2ms/step - loss: 2.0699e-06 - acc: 1.0000
Epoch 96/150
187/187 [==============================] - 0s 2ms/step - loss: 2.0060e-06 - acc: 1.0000
Epoch 97/150
187/187 [==============================] - 0s 2ms/step - loss: 1.9469e-06 - acc: 1.0000
Epoch 98/150
187/187 [==============================] - 0s 2ms/step - loss: 1.8885e-06 - acc: 1.0000
Epoch 99/150
187/187 [==============================] - 0s 2ms/step - loss: 1.8333e-06 - acc: 1.0000
Epoch 100/150
187/187 [==============================] - 0s 2ms/step - loss: 1.7794e-06 - acc: 1.0000
Epoch 101/150
187/187 [==============================] - 0s 2ms/step - loss: 1.7278e-06 - acc: 1.0000
Epoch 102/150
187/187 [==============================] - 0s 2ms/step - loss: 1.6796e-06 - acc: 1.0000


187/187 [==============================] - 0s 2ms/step - loss: 7.7725e-05 - acc: 1.0000
Epoch 24/150
187/187 [==============================] - 0s 1ms/step - loss: 6.8443e-05 - acc: 1.0000
Epoch 25/150
187/187 [==============================] - 0s 1ms/step - loss: 6.1309e-05 - acc: 1.0000
Epoch 26/150
187/187 [==============================] - 0s 1ms/step - loss: 5.5497e-05 - acc: 1.0000
Epoch 27/150
187/187 [==============================] - 0s 1ms/step - loss: 5.0197e-05 - acc: 1.0000
Epoch 28/150
187/187 [==============================] - 0s 1ms/step - loss: 4.5678e-05 - acc: 1.0000
Epoch 29/150
187/187 [==============================] - 0s 2ms/step - loss: 4.1739e-05 - acc: 1.0000
Epoch 30/150
187/187 [==============================] - 0s 2ms/step - loss: 3.8298e-05 - acc: 1.0000
Epoch 31/150
187/187 [==============================] - 0s 2ms/step - loss: 3.5210e-05 - acc: 1.0000
Epoch 32/150
187/187 [==============================] - 0s 1ms/step - loss: 3.2425e-05 - acc: 1.0000
Epo

...

188/188 [==============================] - 0s 2ms/step - loss: 3.7771e-06 - acc: 1.0000
Epoch 78/150
188/188 [==============================] - 0s 2ms/step - loss: 3.6335e-06 - acc: 1.0000
Epoch 79/150
188/188 [==============================] - 0s 2ms/step - loss: 3.4954e-06 - acc: 1.0000
Epoch 80/150
188/188 [==============================] - 0s 2ms/step - loss: 3.3769e-06 - acc: 1.0000
Epoch 81/150
188/188 [==============================] - 0s 2ms/step - loss: 3.2530e-06 - acc: 1.0000
Epoch 82/150
188/188 [==============================] - 0s 2ms/step - loss: 3.1436e-06 - acc: 1.0000
Epoch 83/150
188/188 [==============================] - 0s 2ms/step - loss: 3.0455e-06 - acc: 1.0000
Epoch 84/150
188/188 [==============================] - 0s 2ms/step - loss: 2.9370e-06 - acc: 1.0000
Epoch 85/150
188/188 [==============================] - 0s 2ms/step - loss: 2.8407e-06 - acc: 1.0000
Epoch 86/150
188/188 [==============================] - 0s 2ms/step - loss: 2.7505e-06 - acc: 1.0000
Epo

188/188 [==============================] - 0s 2ms/step - loss: 0.0600 - acc: 0.9840
Epoch 8/150
188/188 [==============================] - 0s 2ms/step - loss: 0.0135 - acc: 1.0000
Epoch 9/150
188/188 [==============================] - 0s 2ms/step - loss: 0.0036 - acc: 1.0000
Epoch 10/150
188/188 [==============================] - 0s 2ms/step - loss: 0.0021 - acc: 1.0000
Epoch 11/150
188/188 [==============================] - 0s 2ms/step - loss: 0.0015 - acc: 1.0000
Epoch 12/150
188/188 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 13/150
188/188 [==============================] - 0s 2ms/step - loss: 8.6488e-04 - acc: 1.0000
Epoch 14/150
188/188 [==============================] - 0s 2ms/step - loss: 7.0148e-04 - acc: 1.0000
Epoch 15/150
188/188 [==============================] - 0s 2ms/step - loss: 5.7511e-04 - acc: 1.0000
Epoch 16/150
188/188 [==============================] - 0s 2ms/step - loss: 4.8392e-04 - acc: 1.0000
Epoch 17/150
188/188 [=======

188/188 [==============================] - 0s 2ms/step - loss: 6.1777e-06 - acc: 1.0000
Epoch 89/150
188/188 [==============================] - 0s 2ms/step - loss: 5.9763e-06 - acc: 1.0000
Epoch 90/150
188/188 [==============================] - 0s 2ms/step - loss: 5.7818e-06 - acc: 1.0000
Epoch 91/150
188/188 [==============================] - 0s 2ms/step - loss: 5.6000e-06 - acc: 1.0000
Epoch 92/150
188/188 [==============================] - 0s 2ms/step - loss: 5.4126e-06 - acc: 1.0000
Epoch 93/150
188/188 [==============================] - 0s 2ms/step - loss: 5.2434e-06 - acc: 1.0000
Epoch 94/150
188/188 [==============================] - 0s 2ms/step - loss: 5.0726e-06 - acc: 1.0000
Epoch 95/150
188/188 [==============================] - 0s 2ms/step - loss: 4.9132e-06 - acc: 1.0000
Epoch 96/150
188/188 [==============================] - 0s 2ms/step - loss: 4.7546e-06 - acc: 1.0000
Epoch 97/150
188/188 [==============================] - 0s 2ms/step - loss: 4.6118e-06 - acc: 1.0000
Epo

188/188 [==============================] - 0s 2ms/step - loss: 1.0497e-04 - acc: 1.0000
Epoch 19/150
188/188 [==============================] - 0s 2ms/step - loss: 8.6265e-05 - acc: 1.0000
Epoch 20/150
188/188 [==============================] - 0s 2ms/step - loss: 7.3152e-05 - acc: 1.0000
Epoch 21/150
188/188 [==============================] - 0s 2ms/step - loss: 6.0928e-05 - acc: 1.0000
Epoch 22/150
188/188 [==============================] - 0s 2ms/step - loss: 5.2103e-05 - acc: 1.0000
Epoch 23/150
188/188 [==============================] - 0s 2ms/step - loss: 4.4144e-05 - acc: 1.0000
Epoch 24/150
188/188 [==============================] - 0s 2ms/step - loss: 3.8320e-05 - acc: 1.0000
Epoch 25/150
188/188 [==============================] - 0s 2ms/step - loss: 3.2830e-05 - acc: 1.0000
Epoch 26/150
188/188 [==============================] - 0s 2ms/step - loss: 2.8095e-05 - acc: 1.0000
Epoch 27/150
188/188 [==============================] - 0s 2ms/step - loss: 2.4032e-05 - acc: 1.0000
Epo

188/188 [==============================] - 0s 2ms/step - loss: 2.4130e-07 - acc: 1.0000
Epoch 99/150
188/188 [==============================] - 0s 2ms/step - loss: 2.3921e-07 - acc: 1.0000
Epoch 100/150
188/188 [==============================] - 0s 2ms/step - loss: 2.3541e-07 - acc: 1.0000
Epoch 101/150
188/188 [==============================] - 0s 2ms/step - loss: 2.2688e-07 - acc: 1.0000
Epoch 102/150
188/188 [==============================] - 0s 2ms/step - loss: 2.2543e-07 - acc: 1.0000
Epoch 103/150
188/188 [==============================] - 0s 2ms/step - loss: 2.1892e-07 - acc: 1.0000
Epoch 104/150
188/188 [==============================] - 0s 2ms/step - loss: 2.1639e-07 - acc: 1.0000
Epoch 105/150
188/188 [==============================] - 0s 2ms/step - loss: 2.1179e-07 - acc: 1.0000
Epoch 106/150
188/188 [==============================] - 0s 2ms/step - loss: 2.0810e-07 - acc: 1.0000
Epoch 107/150
188/188 [==============================] - 0s 2ms/step - loss: 2.0182e-07 - acc: 1.

In [13]:
# tunning my model
def create_larger():
    model=models.Sequential()
    model.add(layers.Dense(60, kernel_initializer='normal',activation='relu',input_shape=(60,)))
    model.add(layers.Dense(40, kernel_initializer='normal',activation='relu'))
    model.add(layers.Dense(10,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='sgd',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=15, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Tunned: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tunned: 76.92% (6.98%)


# keras Functional API

In [14]:
import keras
def functional_api_model():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(60,activation='relu')(inputs)
    x=layers.Dense(40,activation='relu')(x)
    outputs = layers.Dense(1,activation='sigmoid')(x)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [15]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=functional_api_model, epochs=15, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Tunned: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tunned: 80.66% (5.93%)


In [32]:
results

array([0.90909091, 0.80952382, 0.71428573, 0.85714287, 0.85714287,
       0.85714287, 0.76190477, 0.8       , 0.75000001, 0.75000001])

# Model subclassing

In [16]:
class MyModel(keras.Model):
    
    def _init_(self):
        super(MyModel, self)._init_()
        self.dense1=layers.Dense(60,activation='relu')
        self.dense2=layers.Dense(30,activation='relu')
        self.dense3=layers.Dense(1,activation='sigmoid')
        
    def call(self,inputs):
        x=self.dense1(inputs)
        x=self.dense2(x)
        return self.dense3(x)

model = MyModel()



In [18]:
estimator = KerasClassifier(build_fn=MyModel(), epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

NotImplementedError: 

#train model without scikit learn

In [22]:
from keras import models
from keras import layers
def build_model():
    model=models.Sequential()
    model.add(layers.Dense(60,activation='relu',input_shape=(60,)))
    model.add(layers.Dense(30,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model



In [33]:
import numpy as np
k = 10
num_val_samples = len(X) // k
num_epochs = 10
all_mae_histories = []

for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
    [X[:i * num_val_samples],
    X[(i + 1) * num_val_samples:]],
    axis=0)
    partial_train_targets = np.concatenate(
    [encoded_y[:i * num_val_samples],
    encoded_y[(i + 1) * num_val_samples:]],
    axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
    validation_data=(val_data, val_targets),
    epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history
    all_mae_histories.append(mae_history)


processing fold # 0
Train on 188 samples, validate on 20 samples
Epoch 1/10
188/188 [==============================] - 7s 38ms/step - loss: 0.6298 - acc: 0.6436 - val_loss: 0.6693 - val_acc: 0.6500
Epoch 2/10
188/188 [==============================] - 1s 8ms/step - loss: 0.5335 - acc: 0.7340 - val_loss: 0.6975 - val_acc: 0.6000
Epoch 3/10
188/188 [==============================] - 1s 8ms/step - loss: 0.4773 - acc: 0.7606 - val_loss: 1.0013 - val_acc: 0.4500
Epoch 4/10
188/188 [==============================] - 1s 8ms/step - loss: 0.4134 - acc: 0.8245 - val_loss: 0.9695 - val_acc: 0.5000
Epoch 5/10
188/188 [==============================] - 1s 8ms/step - loss: 0.3586 - acc: 0.8298 - val_loss: 1.2025 - val_acc: 0.4000
Epoch 6/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3371 - acc: 0.8777 - val_loss: 1.9476 - val_acc: 0.1500
Epoch 7/10
188/188 [==============================] - 1s 8ms/step - loss: 0.3152 - acc: 0.8511 - val_loss: 1.3366 - val_acc: 0.4000
Epoch 8/10

Epoch 10/10
188/188 [==============================] - 1s 8ms/step - loss: 0.3228 - acc: 0.8564 - val_loss: 2.6268 - val_acc: 0.0000e+00
processing fold # 6
Train on 188 samples, validate on 20 samples
Epoch 1/10
188/188 [==============================] - 8s 40ms/step - loss: 0.6701 - acc: 0.6170 - val_loss: 0.8881 - val_acc: 0.0000e+00
Epoch 2/10
188/188 [==============================] - 1s 8ms/step - loss: 0.6214 - acc: 0.6755 - val_loss: 0.8171 - val_acc: 0.2000
Epoch 3/10
188/188 [==============================] - 2s 8ms/step - loss: 0.5530 - acc: 0.7340 - val_loss: 0.7585 - val_acc: 0.4500
Epoch 4/10
188/188 [==============================] - 2s 8ms/step - loss: 0.4940 - acc: 0.7340 - val_loss: 0.6580 - val_acc: 0.7500
Epoch 5/10
188/188 [==============================] - 2s 8ms/step - loss: 0.4383 - acc: 0.7979 - val_loss: 0.8966 - val_acc: 0.4000
Epoch 6/10
188/188 [==============================] - 2s 9ms/step - loss: 0.4244 - acc: 0.7926 - val_loss: 0.6366 - val_acc: 0.7000
E

AttributeError: 'list' object has no attribute 'values'